### 1. Import Libraries

In [10]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from itertools import combinations

### 2. Data Obtention

In [11]:
# Lista de empresas a evaluar
tickers_list = ['VALE', 'BHP', 'RIO', 'SCCO', 'BTU', 'AMR', 'AES', 'UNP', 'CNI']

for ticker in tickers_list:
    data = yf.Ticker(ticker).history(start='2023-01-01', end='2023-12-31')
    if data.empty:
        print(f"no hay datos para {ticker}")
    else:
        print(f"{ticker} tiene datos")

# Descargar datos históricos
tickers = yf.Tickers(" ".join(tickers_list))
hist = tickers.history(start='2021-01-01',end='2024-12-31')

print(hist['Close'].head())

adj_close = hist['Close']

#rellenar con el último valor disponible para los vacíos
#adj_close = hist['Close'].fillna(method='ffill')
#adj_close = adj_close.fillna(method='bfill')  


if adj_close.empty:
    raise ValueError("no se obtuvieron datos de precios de estos activos")

print(adj_close)

VALE tiene datos
BHP tiene datos
RIO tiene datos
SCCO tiene datos
BTU tiene datos
AMR tiene datos
AES tiene datos
UNP tiene datos
CNI tiene datos


[*********************100%***********************]  9 of 9 completed

Ticker            AES  AMR        BHP       BTU         CNI        RIO  \
Date                                                                     
2021-01-04  20.197660  NaN  51.894207  2.860808  105.250114  54.546722   
2021-01-05  20.875549  NaN  53.550819  3.337609  106.662041  55.864761   
2021-01-06  21.709871  NaN  55.253662  3.201380  108.506165  58.359848   
2021-01-07  21.492601  NaN  56.386311  3.181919  109.236137  60.601196   
2021-01-08  22.092270  NaN  56.602058  3.123535  111.474075  61.101631   

Ticker           SCCO         UNP       VALE  
Date                                          
2021-01-04  54.866444  185.846649  11.780221  
2021-01-05  55.592335  188.126129  11.976559  
2021-01-06  56.236679  192.062607  12.294760  
2021-01-07  58.055492  194.735779  12.816067  
2021-01-08  58.267551  200.320038  12.809298  
Ticker            AES         AMR        BHP        BTU         CNI  \
Date                                                                  
2021-01-04

### 3. Portfolios Analysis

In [12]:

# Función para calcular métricas del portafolio
def calcular_metricas(adj_close_values):
    R = np.log(adj_close_values[1:] / adj_close_values[:-1])  # Retornos logarítmicos
    RE = np.mean(R, axis=0) * 252  # Retorno esperado anualizado
    RI = np.std(R, axis=0) * np.sqrt(252)  # Riesgo anualizado
    S = np.cov(R, rowvar=False)  # Matriz de covarianza
    corr = np.corrcoef(R, rowvar=False)  # Matriz de correlación
    return RE, RI, S, corr

# Obtener métricas generales
adj_close_values = adj_close.values
RE, RI, S, correlation_matrix = calcular_metricas(adj_close_values)

# Evaluar todas las combinaciones posibles de 4 activos
mejores_portafolios = []
for subset in combinations(adj_close.columns, 4):
    indices = [adj_close.columns.get_loc(ticker) for ticker in subset]
    corr_submatrix = correlation_matrix[np.ix_(indices, indices)]

    # Filtrar combinaciones con correlación > 0.5
    if np.any(np.abs(np.triu(corr_submatrix, k=1)) > 0.5):
        continue

    # Extraer datos de la combinación aceptable
    RE_sub = RE[indices]
    S_sub = S[np.ix_(indices, indices)]
    weights = np.ones(4) / 4  # Pesos iniciales iguales

    # Definir restricciones y límites
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, None)] * 4

    # Optimizar el portafolio
    res = minimize(lambda w: w @ S_sub @ w.T, x0=weights, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12})

    if res.success:
        # Calcular métricas del portafolio optimizado
        ReP = res.x @ RE_sub.T
        varP = res.x @ S_sub @ res.x.T
        RiP = np.sqrt(varP)
        SharpeP = ReP / RiP

        # Guardar resultados
        mejores_portafolios.append({
            "Activos": subset,
            "ReP (%)": round(ReP * 100, 4),
            "RiP (%)": round(RiP * 100, 4),
            "SharpeP": round(SharpeP, 4)
        })

# Ordenar por el mejor Sharpe Ratio
mejores_portafolios = sorted(mejores_portafolios, key=lambda x: x["SharpeP"], reverse=True)

# Convertir a DataFrame y mostrar los resultados
df_resultados = pd.DataFrame(mejores_portafolios)
print(df_resultados)

# Guardar en un archivo Excel
df_resultados.to_excel("mejores_portafolios.xlsx", index=False)
print("\nResultados guardados en 'mejores_portafolios.xlsx'")

                 Activos  ReP (%)  RiP (%)  SharpeP
0  (AES, BTU, SCCO, UNP)   3.6935   1.3156   2.8074
1   (AES, BTU, RIO, UNP)   2.1106   1.2587   1.6769
2   (AES, BHP, BTU, UNP)   1.3807   1.2817   1.0772
3  (AES, BTU, UNP, VALE)   0.5063   1.2888   0.3928
4  (AES, BTU, CNI, SCCO)  -0.8841   1.3336  -0.6630
5   (AES, BTU, CNI, RIO)  -1.8293   1.2832  -1.4255
6   (AES, BHP, BTU, CNI)  -2.6673   1.3035  -2.0462
7  (AES, BTU, CNI, VALE)  -3.2014   1.3089  -2.4458

Resultados guardados en 'mejores_portafolios.xlsx'
